# Plot Idealized Channel Grid

This notebook demonstrates how to plot an idealized rectangular channel mesh from `chn_grd.dat` using xfvcom.
The grid uses Cartesian coordinates in meters without any geographic information.

In [ ]:
# Import required libraries
import matplotlib.pyplot as plt
import matplotlib.tri as mtri
import numpy as np
from pathlib import Path

# Import xfvcom components
from xfvcom import FvcomInputLoader, FvcomPlotter, FvcomPlotConfig, FvcomPlotOptions

## Using FvcomInputLoader for Grid-Only Plotting

With the new `FvcomInputLoader`, we can load input files and create datasets compatible with `FvcomPlotter`. This allows us to use the same plotting tools for both input and output data.

In [ ]:
# Path to the grid file
grid_file = Path("input/chn_grd.dat")

# Load the grid using FvcomInputLoader
# For idealized grids, we use a dummy UTM zone (the generated lon/lat won't be meaningful)
loader = FvcomInputLoader(
    grid_path=grid_file, 
    utm_zone=54,
    add_dummy_time=True,
    add_dummy_siglay=True
)

# Display grid information
print(f"Grid loaded from: {loader.grid_path}")
print(f"Number of nodes: {loader.grid.node}")
print(f"Number of elements: {loader.grid.nele}")
print(f"X range: {loader.grid.x.min():.0f} to {loader.grid.x.max():.0f} meters")
print(f"Y range: {loader.grid.y.min():.0f} to {loader.grid.y.max():.0f} meters")
print(f"\nDataset dimensions: {dict(loader.ds.sizes)}")
print(f"Dataset variables: {list(loader.ds.data_vars)}")
print("\nNote: This is an idealized grid. We will use Cartesian coordinates (x,y).")

## Plot the mesh using FvcomPlotter

Now we can use the standard `FvcomPlotter` with our input dataset, just like we would with FVCOM output data.

In [ ]:
# Create plot configuration and plotter
config = FvcomPlotConfig()
plotter = FvcomPlotter(loader.ds, config)

# Create plot options for Cartesian mode
opts = FvcomPlotOptions(
    use_latlon=False,  # Use Cartesian coordinates for idealized grid
    with_mesh=True,    # Show the triangular mesh
    coastlines=False,  # No coastlines for idealized grid
    obclines=False,    # No open boundary lines
    mesh_linewidth=0.5,
    mesh_color='black',
    figsize=(12, 6)
)

# Plot the mesh only (da=None means plot mesh only)
ax = plotter.plot_2d(da=None, opts=opts)

# Add labels and title
ax.set_xlabel('X (meters)')
ax.set_ylabel('Y (meters)')
ax.set_title('Idealized Channel Grid - Using FvcomPlotter')

# Add grid for better visualization
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate element areas using the grid from loader
grid = loader.grid

# Get node coordinates for each element
x_elem = grid.x[grid.nv]  # Shape: (3, nele)
y_elem = grid.y[grid.nv]  # Shape: (3, nele)

# Calculate areas using cross product
# Area = 0.5 * |AB × AC| where A, B, C are the three vertices
areas = 0.5 * np.abs(
    (x_elem[1] - x_elem[0]) * (y_elem[2] - y_elem[0]) - 
    (x_elem[2] - x_elem[0]) * (y_elem[1] - y_elem[0])
)

print(f"Element area statistics:")
print(f"  Min area: {areas.min():.2e} m²")
print(f"  Max area: {areas.max():.2e} m²")
print(f"  Mean area: {areas.mean():.2e} m²")
print(f"  Total area: {areas.sum():.2e} m²")

## Plot zoomed views using FvcomPlotter

We can use the `xlim` and `ylim` options to create zoomed views of the mesh.

In [ ]:
# Create two plots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Full mesh
opts_full = FvcomPlotOptions(
    use_latlon=False,
    with_mesh=True,
    coastlines=False,
    obclines=False,
    mesh_linewidth=0.3,
    mesh_color='black'
)
plotter.plot_2d(da=None, ax=ax1, opts=opts_full)
ax1.set_xlabel('X (meters)')
ax1.set_ylabel('Y (meters)')
ax1.set_title('Full Channel Mesh')
ax1.grid(True, alpha=0.3)

# Zoomed view
zoom_x = [100000, 200000]  # 100 km section
zoom_y = [150000, 250000]  # 100 km section

opts_zoom = FvcomPlotOptions(
    use_latlon=False,
    with_mesh=True,
    coastlines=False,
    obclines=False,
    mesh_linewidth=0.5,
    mesh_color='black',
    xlim=zoom_x,
    ylim=zoom_y
)
plotter.plot_2d(da=None, ax=ax2, opts=opts_zoom)
ax2.set_xlabel('X (meters)')
ax2.set_ylabel('Y (meters)')
ax2.set_title('Zoomed View of Mesh')
ax2.grid(True, alpha=0.3)

# Add rectangle on full mesh to show zoom area
from matplotlib.patches import Rectangle
rect = Rectangle((zoom_x[0], zoom_y[0]), 
                 zoom_x[1] - zoom_x[0], 
                 zoom_y[1] - zoom_y[0],
                 fill=False, edgecolor='red', linewidth=2)
ax1.add_patch(rect)

plt.tight_layout()
plt.show()

## Save the plot

In [ ]:
# Create a high-quality plot for saving
opts_hq = FvcomPlotOptions(
    use_latlon=False,
    with_mesh=True,
    coastlines=False,
    obclines=False,
    mesh_linewidth=0.3,
    mesh_color='navy',
    figsize=(14, 7)
)

ax = plotter.plot_2d(da=None, opts=opts_hq)

ax.set_xlabel('X (meters)', fontsize=12)
ax.set_ylabel('Y (meters)', fontsize=12)
ax.set_title('Idealized Rectangular Channel Grid - FVCOM Mesh', fontsize=14)
ax.grid(True, alpha=0.2, linestyle='--')

# Save the figure
plt.tight_layout()
plt.savefig('chn_mesh.png', dpi=300, bbox_inches='tight')
print("Mesh plot saved as 'chn_mesh.png'")

plt.show()